Imports

In [123]:
import pandas as pd
import re

Loading and Pre-processing Profanity + English Dictionary Datasets

In [73]:
#loads and creates a dataframe from a .txt list of profanities
profanity_data1 = []
with open('profanity_data.txt') as f:
    lines = f.readlines()
    for i in lines:
        profanity_data1.append(i[:-1])
    f.close()
profanity_data1 = pd.DataFrame(profanity_data1)
profanity_data1 = profanity_data1.rename(columns={0:"Word"})

In [74]:
#loads a creates a dataframe from a .csv of profanities
profanity_data2 = pd.read_csv("bad-words.csv", header=None)
profanity_data2 = profanity_data2.rename(columns={0:"Word"})

In [75]:
#combines the two dataframes and creates a binary variable that indicates whether a word is a profanity or not (1 or 0)
profanity_df = pd.concat([profanity_data1, profanity_data2], join="inner")
profanity_df = profanity_df.drop_duplicates()
profanity_df["Profanity_indicator"] = 1

In [76]:
#loads dictionary dataset
dictionary_data = pd.read_csv("dictionary.csv", header=None)
dictionary_data = dictionary_data.rename(columns={0:"Word"})
dictionary_data = dictionary_data.drop(columns=[1, 2])

#removes any words with less than 3 characters, with empty spaces, and more than 10 characters
def remove_word(df):
    
    if len(df["Word"]) <=2 or " " in df["Word"] or len(df["Word"]) >= 10:
        return 1
    else:
        return 0
dictionary_data["remove_word"] = dictionary_data.apply(remove_word, axis=1)
dictionary_data = dictionary_data[dictionary_data["remove_word"] == 0]
dictionary_data = dictionary_data.drop(columns=["remove_word"])

#gets a random sample of 4000 words to use later in training/testing
dictionary_data = dictionary_data.sample(4000)

#assigns profanity_indicator to 0
dictionary_data["Profanity_indicator"] = 0

In [133]:
#combines the profanity and dictionary dataframes
words_dataset = pd.concat([profanity_df, dictionary_data])

#creates addition variables to use as features

#length of word
words_dataset["word_length"] = words_dataset['Word'].str.len()

#indicator of whether word is longer than avg
avg_word_len = words_dataset["word_length"].mean()
def longer_than_avg(df):
    if df["word_length"] >= avg_word_len:
        return 1
    else:
        return 0   
words_dataset["longer_than_avg"] = words_dataset.apply(longer_than_avg, axis=1)

#indicator of whether word has numerics
def num_numerics(df):
    bool_numeric = re.search(r'\d', df["Word"])
    if bool_numeric != None:
        return 1
    else:
        return 0
words_dataset["num_in_string"] = words_dataset.apply(num_numerics, axis=1)

words_dataset

,Word,Profanity_indicator,word_length,longer_than_avg,num_in_string
0,4r5e,1,4,0,1
1,5h1t,1,4,0,1
2,5hit,1,4,0,1
3,a55,1,3,0,1
4,anal,1,4,0,0
...,...,...,...,...,...
34638,Quackling,0,9,1,0
42430,Rotal,0,5,0,0
13837,Fusteric,0,8,1,0
15365,Genu,0,4,0,0
